# 趋势策略入门

### 关键函数：
 - 1 交易函数（接收一个交易字典，包含交易价格，交易量，币种，交易方向等）
 - 2 持仓计算函数（根据仓位情况和风险设定，计算限制交易量以及交易价格）
 - 3 趋势条件计算函数（根据品种价格情况，判断是否执行交易，交易方向，交易量，以及交易价格）

In [ ]:
from sklearn.linear_model import LinearRegression

In [1]:
class mid_class():
    def __init__(self, this_exchange):
        '''
        初始化数据填充交易所的信息，首次获取价格，首次获取account信息
        设定好密钥……
        
        Args:
            this_exchange: FMZ的交易所结构
        
        '''
        self.init_timestamp = time.time()
        self.exchange = this_exchange
        self.name = self.exchange.GetName()
        self.jyd = self.exchange.GetCurrency()        
    
    def get_account(self):
        '''
        获取账户信息
        
        Returns:
            获取信息成功返回True，获取信息失败返回False
        '''
        self.Balance = '---'
        self.Amount = '---'
        self.FrozenBalance = '---'
        self.FrozenStocks = '---'
        
        try:
            self.account = self.exchange.GetAccount()

            self.Balance =  self.account['Balance']
            self.Amount = self.account['Stocks']
            self.FrozenBalance =  self.account['FrozenBalance']
            self.FrozenStocks = self.account['FrozenStocks']
            return True
        except:
            return False
    
    def get_ticker(self):
        '''
        获取市价信息
        
        Returns:
            获取信息成功返回True，获取信息失败返回False
        '''
        self.high = '---'
        self.low = '---'
        self.Sell =  '---'
        self.Buy =  '---'
        self.last =  '---'
        self.Volume = '---'
        
        try:
            self.ticker = self.exchange.GetTicker()
        
            self.high = self.ticker['High']
            self.low = self.ticker['Low']
            self.Sell =  self.ticker['Sell']
            self.Buy =  self.ticker['Buy']
            self.last =  self.ticker['Last']
            self.Volume = self.ticker['Volume']
            return True
        except:
            return False
        
        
    def get_depth(self):
        '''
        获取深度信息
        
        Returns:
            获取信息成功返回True，获取信息失败返回False
        '''
        self.Ask = '---'
        self.Bids = '---'
        
        try:
            self.Depth = self.exchange.GetDepth()
            self.Ask = self.Depth['Asks']
            self.Bids = self.Depth ['Bids']
            return True
        except:
            return False
        
        
    
    def get_ohlc_data(self, period = PERIOD_M5):
        '''
        获取K线信息
        
        Args:
            period: K线周期，PERIOD_M1 指1分钟, PERIOD_M5 指5分钟, PERIOD_M15 指15分钟,
            PERIOD_M30 指30分钟, PERIOD_H1 指1小时, PERIOD_D1 指一天。
        '''
        self.ohlc_data = exchange.GetRecords(period)
        
        
    
    def create_order(self, order_type, price, amount):
        '''
        post一个挂单信息
        
        Args:
            order_type：挂单类型，'buy'指挂买单，'sell'指挂卖单
            price：挂单价格
            amount:挂单数量
            
        Returns:
            挂单Id号，可用以取消挂单
        '''
        if order_type == 'buy':
            try:
                order_id = self.exchange.Buy( price, amount)
            except:
                return False
            
        elif order_type == 'sell':
            try:
                order_id = self.exchange.Sell( price, amount)
            except:
                return False
        
        return order_id
    
    def get_orders(self):
        self.undo_ordes = self.exchange.GetOrders()
        return self.undo_ordes
    
    def cancel_order(self, order_id):
        '''
        取消一个挂单信息
        
        Args:
            order_id：希望取消的挂单ID号
            
        Returns:
            取消挂单成功返回True，取消挂单失败返回False
        '''
        return self.exchange.CancelOrder(order_id)
        
    def refreash_data(self):
        '''
        刷新信息
        
        Returns:
            刷新信息成功返回 'refreash_data_finish!' 否则返回相应刷新失败的信息提示
        '''

        if not self.get_account():
            return 'false_get_account'
        
        if not self.get_ticker():
            return 'false_get_ticker'
        if not self.get_depth():
            return 'false_get_depth'
        try:
            self.get_ohlc_data()
        except:
            return 'false_get_K_line_info'
        
        return 'refreash_data_finish!'


NameError: name 'PERIOD_M5' is not defined

In [3]:
def make_MACD(Kline, short_period, long_period, mid_period):
    DIFF = make_DIF(Kline, short_period, long_period)
    DEA = make_DEA(DIFF, mid_period)
    MACDs = []
    for i in range(len(DEA)):
        MACDs.append( 2*(DEA[i] - DIFF[i]) )
    return MACDs
    
def make_DIF(Kline, short_period, long_period):
    EMA_short = make_EMA(Kline, short_period)
    EMA_long = make_EMA(Kline, long_period)
    DIFs = []
    the_len = len(EMA_short)
    for i in range( the_len ):
        DIFs.append( EMA_short[ i - the_len ] - EMA_long[ i-the_len ])
    return DIFs

def make_DEA(DIFF, mid_period):
    return make_EMA(DIFF, mid_period)
    
def make_EMA(Kline, period):
    this_Kline = Kline[-period:]
    Log(this_Kline)
    EMAs = []
    try:
        EMAs.append( this_Kline[0]['Close'] )
        for i in range(len(this_Kline) -1 ):
            EMAs.append( ( 2*this_Kline[i+1]['Close'] + (period -1)*EMAs[i] )/(period + 1) )
    except:
        EMAs.append( this_Kline[0] )
        for i in range(len(this_Kline) -1 ):
            EMAs.append( ( 2*this_Kline[i+1] + (period -1)*EMAs[i] )/(period + 1) )
            
    return EMAs

class qushi_class():
    def __init__(self, mid_class, amount_N, price_N):
        '''
        设定好初始需要考虑的参数
        Args:
            mid_class: 所使用的交易所中间层
            amount_N：数量小数点限制
            price_N：价格小数点限制
            
        Attributes：
            amount_N：数量小数点限制
            price_N：价格小数点限制
            init_time：初始时间
            last_time：上一次执行操作的时间
            trade_list:交易请求的id
        '''
        self.jys = mid_class
        
        self.init_time = time.time()
        self.last_time = time.time()
        
        self.amount_N = amount_N
        self.price_N = price_N
        
        self.trade_list = []
    
    def refreash_data(self):
        '''
        用来从交易所获取最新的价格和数量信息
        
        Attributes：
            B：商品币数量
            money：计价币数量
            can_buy_B：当前理论可购买商品币数量
            Buy_price:当前市场上最近的一单挂单买价
            Sell_price：当前市场上最近的一单挂单卖价
        '''
        
        self.jys.refreash_data()
        self.B = self.jys.Amount
        self.money = self.jys.Balance
        self.Buy_price = self.jys.Buy
        self.Sell_price = self.jys.Sell
        self.can_buy_B = self.money/ self.Sell_price
        self.can_buy_B = _N(self.can_buy_B, self.amount_N )
        self.can_sell_B = _N(self.B, self.amount_N )
        
    def make_trade_by_dict(self, trade_dicts):
        '''
        用来批量完成交易订单
        
        Attributes：
            trade_list:已提交的交易请求的id
        '''
        for this_trade in trade_dicts:
            this_price = _N(this_trade['price'], self.price_N )
            this_amount = _N(this_trade['amount'], self.amount_N )
            
            this_trade_id = self.jys.create_order( this_trade['side'], this_price , this_amount ) 
            self.trade_list.append( this_trade_id )
    
    def condition_chicang(self, hands_num):
        '''
        根据持仓情况来做交易判定的条件
        Args:
            hands_num：表示交易一共几手（我们假设当前每次交易不高于一手）
            
        Attributes：
            min_trade_B: 一手最多交易的商品币数量
            min_trade_money: 一手最多交易的计价币数量
        
        '''
        self.min_trade_B = (self.can_buy_B + self.B) / hands_num
        self.min_buy_B = min(self.min_trade_B, self.can_buy_B)
        self.min_sell_B = min(self.min_trade_B, self.B)
        
        self.min_trade_money = self.min_trade_B* self.jys.Buy


    
    def condition_qushi(self, macd_threshold, short_period = 12, long_period = 26, mid_period = 9 ):
        '''
        根据市场价格情况来做交易判定的条件
        Args:
            short_period：MACD的EMA短周期
            long_period:MACD的EMA长周期
            mid_period:DIF的EMA周期
            macd_threshold:MACD预计变化百分之多少取信
        Returns：
            min_trade_B: 一手最多交易的商品币数量
            min_trade_money: 一手最多交易的计价币数量
        
        '''
        Kline = self.jys.ohlc_data
        MACD = make_MACD(Kline, short_period, long_period, mid_period)
        X = [[x+1,x+1] for x in range(mid_period)]
        y = MACD
        reg = LinearRegression().fit(X, y)
        next_macd = reg.predict( [[ mid_period + 1,mid_period + 1]] )
        mean_macd = sum(MACD)/len(MACD)
        
        more_than = (100+macd_threshold)/100
        less_than = (100-macd_threshold)/100
        
        rt = False
        if  next_macd> macd_threshold and MACD [0] <0 :
            rt = 'Buy'
        elif next_macd< -macd_threshold and MACD [0] >0:
            rt = 'Sell'
        
        return rt
    
    
    def make_trade_dicts(self, hands_num, change_pct ):
        '''
        制作交易用字典表单
        Args:
            hands_num：一共交易多少手
            change_pct：价格变化多少交易一手
            
        Returns：
            this_trade_dicts: 根据当前价格变化，制作出需交易的字典的列表
        
        '''
        self.condition_chicang(hands_num)
        rt = self.condition_qushi( change_pct )
        this_trade_dicts = []
        if rt:
            if rt == 'Buy':
                if self.min_buy_B > 10**-self.amount_N:
                    this_trade_dicts.append({
                        'side':'buy',
                        'price':self.jys.Buy,
                        'amount':self.min_buy_B
                    })
            else:
                if self.min_sell_B > 10**-self.amount_N:
                    this_trade_dicts.append({
                        'side':'sell',
                        'price':self.jys.Sell,
                        'amount':self.min_sell_B
                    })
            return this_trade_dicts
        else:
            return False
        

In [1]:
def main():
    
    Set_amount_N = 4
    Set_price_N = 4
    
    hands_num = 20
    macd_threshold = 10
    
    test_mid = mid_class(exchange)
    Log(test_mid.refreash_data())
    test_qushi = qushi_class(test_mid , Set_amount_N, Set_price_N)
    
    while True:
        
        Sleep(1000)
        try:
            test_qushi.refreash_data()

            now_trade_dicts = test_qushi.make_trade_dicts(hands_num, macd_threshold)
            if now_trade_dicts:
                test_qushi.make_trade_by_dict(now_trade_dicts)
                now_trade_dicts = False
        except:
            pass

## 优化策略的思路
    作出假设 --> 验证假设 --> 写成函数投入策略
    
## 优化策略的方法
    1、优化价格判定条件
    2、优化持仓判定条件
    3、优化交易表单的制作函数

In [5]:
def make_MACD(Kline, short_period, long_period, mid_period):
    DIFF = make_DIF(Kline, short_period, long_period)
    DEA = make_DEA(DIFF, mid_period)
    MACDs = []
    for i in range(len(DEA)):
        MACDs.append( 2*(DEA[i] - DIFF[i]) )
    return MACDs
    
def make_DIF(Kline, short_period, long_period):
    EMA_short = make_EMA(Kline, short_period)
    EMA_long = make_EMA(Kline, long_period)
    DIFs = []
    the_len = len(EMA_short)
    for i in range( the_len ):
        DIFs.append( EMA_short[ i - the_len ] - EMA_long[ i-the_len ])
    return DIFs

def make_DEA(DIFF, mid_period):
    return make_EMA(DIFF, mid_period)
    
def make_EMA(Kline, period):
    EMAs = Kline[0]['Close']
    for i in range(len(Kline) -1 ):
        EMAs.append( 2*Kline(i+1)['Close'] + (period -1)*EMAs[i] )/(period + 1)
    return EMAs

[0, 1, 2, 3, 4, 5, 6, 7, 8]

## 所有看起来有效的网格策略，都是在用长期风险换取短期收益 （看起来一直在赚钱，突然有一天本都亏没了）
## 大部分的趋势策略（诸如MACD，EMA这类的简单判定）用近期风险换取长期收益 （看起来一直在亏钱，突然有一天赚了一点点，然后继续亏）

# 人们都容易忽略长期风险，因为短期收益而欢喜

## 作出假设 --> 验证假设 --> 写成函数投入策略
